In [33]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from numpy import array
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import optimizers

In [2]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv('auto-mpg.data', names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.shape

(398, 8)

Clean Data

In [3]:
dataset.isna().sum()
dataset = dataset.dropna()

In [4]:
origin = dataset.pop('Origin')
dataset['USA'] = (origin == 1)*1.0
dataset['Europe'] = (origin == 2)*1.0
dataset['Japan'] = (origin == 3)*1.0

In [14]:
#dataset.shape
#dataset.head()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,USA,Europe,Japan
0,18.0,8,307.0,130.0,3504.0,12.0,70,1.0,0.0,0.0
1,15.0,8,350.0,165.0,3693.0,11.5,70,1.0,0.0,0.0
2,18.0,8,318.0,150.0,3436.0,11.0,70,1.0,0.0,0.0
3,16.0,8,304.0,150.0,3433.0,12.0,70,1.0,0.0,0.0
4,17.0,8,302.0,140.0,3449.0,10.5,70,1.0,0.0,0.0


Split Data

In [5]:
X = pd.DataFrame(dataset.drop("MPG", axis=1))
y = pd.DataFrame(dataset["MPG"])

print(X.shape, y.shape)

(392, 9) (392, 1)


In [91]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [92]:
X_train.shape
y_train.shape

(313, 1)

In [100]:
X_test.shape
y_test.shape

(79, 1)

Scale Data using StandardScaler instead of using the mean and standard deviation
Create a StandardScater model and fit it to the training data

In [9]:
#from sklearn.preprocessing import StandardScaler

#X_scaler = StandardScaler().fit(X_train)

In [10]:
# Transform the training and testing data using the X_scaler

#X_train_scaled = X_scaler.transform(X_train)
#X_test_scaled = X_scaler.transform(X_test)

In [101]:
train_stats = X_train.describe()
train_stats = train_stats.transpose()

In [102]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']

In [103]:
X_train_normed= norm(X_train)
X_test_normed = norm(X_test)

In [59]:
#from sklearn.preprocessing import MinMaxScaler

#scalarX= MinMaxScaler().fit(X_train)

In [60]:
#X_train_scaled = X_scaler.transform(X_train)
#X_test_scaled = X_scaler.transform(X_test)

It is good practice to normalize features that use different scales and ranges. Although the model might converge without 
feature normalization, it makes training more difficult, and it makes the resulting model dependent on the choice 
of units used in the input.

Note: Although we intentionally generate these statistics from only the training dataset, these statistics will also be 
used to normalize the test dataset. We need to do that to project the test dataset into the same distribution that the
model has been trained on.

Test Data is also normalized with training data only

In [104]:
# define and fit the final model
model8 = Sequential()

In [105]:
model8.add(Dense(64, input_dim=9, activation='relu'))
model8.add(Dense(64, activation='relu'))
model8.add(Dense(1, activation='linear'))

In [106]:
model8.compile(loss='mse', optimizer= 'adam', metrics=['mean_absolute_error', 'mean_squared_error'])
model8.fit(X_train_normed, y_train, epochs=1000, verbose=0)

Train Data

In [107]:
loss, mae, mse = model8.evaluate(X_test_normed, y_test, verbose=0)

print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))

Testing set Mean Abs Error:  1.80 MPG
